In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools

Using TensorFlow backend.


In [2]:
first_simulation = simulation.simulator()

Instructions for updating:
Colocations handled automatically by placer.
Filling experience replay memory...
 Retrying in 1 seconds
Started connection for worker # lonely_worker
Stopped connection for worker # lonely_worker
...Done


In [3]:
#first_simulation.evaluate()

In [4]:
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
target_update_frequency = [10, 50, 100]
gamma = [0.6, 0.7, 0.8, 0.9]
eps = [0.1, 0.2]

param_grid = itertools.product(batch_size, target_update_frequency, gamma, eps)

In [ ]:
test = tools.gridsearch(param_grid)

Started process # 1
Started with position 1 and parameters (10, 10, 0.6, 0.1)
1 reached first assignment
Unordered results:


In [ ]:
# agent = simulation.simulator(connection_label = 1)

In [ ]:
# agent.ddqn.train(env = agent.env, num_episodes = 2, policy = "epsGreedy", eps = 0.1)